# Goals

* Add already-processed SRX accessions to the database
  * Datasets already processed by Chris

In [1]:
import os
import warnings
from importlib import resources
import psycopg2
from psycopg2.extensions import connection
import pandas as pd
from dynaconf import Dynaconf
from pypika import Query, Table, Field, Column, Table

In [6]:
from dotenv import load_dotenv
load_dotenv()
os.environ["DYNACONF"] = "test"
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

# Connect

In [11]:
def db_connect() -> connection:
    """Connect to the sql database"""
    s_path = None
    with resources.path("SRAgent", "settings.yml") as settings_path:
        s_path = str(settings_path)
    settings = Dynaconf(
        settings_files=["settings.yml", s_path], 
        environments=True, 
        env_switcher="DYNACONF"
    )
    # connect to db
    db_params = {
        'host': settings.db_host,
        'database': settings.db_name,
        'user': settings.db_user,
        'password': os.environ["GCP_SQL_DB_PASSWORD"],
        'port': settings.db_port,
        'connect_timeout': settings.db_timeout
    }
    return psycopg2.connect(**db_params)

conn = db_connect()

/tmp/ipykernel_945458/3912594006.py:4: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path("SRAgent", "settings.yml") as settings_path:


In [15]:
srx_metadata = Table("srx_metadata")
stmt = Query \
    .from_(srx_metadata) \
    .select("*") 
pd.read_sql(str(stmt), conn).head()

,id,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line
0,11,sra,35087715,SRX25994842,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,human,subcutaneous adipose tissue,breast cancer-related lymphedema,injection of adipose-derived regenerative cell...,adipose derived regenerative cells/stromal vas...
1,14,sra,36178506,ERX11887200,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,mouse,lung tumor,murine lung cancer,murine recombinant coronavirus vector (mCOV) t...,Lewis lung carcinoma (LLC) expressing LCMV gly...
2,15,sra,30749595,SRX22716300,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,human,bone marrow,no,no treatment,primary cells (CD34+ cells)
3,20,sra,18060880,SRX13201194,yes,no,no,not_applicable,not_applicable,not_applicable,mouse,bone marrow,not specified,Notch1+/- genotype,Common Lymphoid Progenitor (CLP)
4,21,sra,36106630,SRX26727599,yes,no,yes,not_applicable,not_applicable,not_applicable,mouse,spinal cord,experimental autoimmune encephalomyelitis (EAE...,EAE_vehicle (control treatment),other


# Load SRX values

In [18]:
infile = "../data/counted_SRX.txt"
counted = pd.read_csv(infile, sep='\t', header=None)
counted.columns = ["srx"]
counted

,srx
0,SRX13670569
1,SRX19828711
2,SRX19829151
3,SRX9556597
4,SRX19826284
...,...
16708,SRX9770752
16709,SRX11523521
16710,SRX19830191
16711,SRX19825307
